# RAG Fundamentals: Building Your First Retrieval-Augmented Generation System

## Table of Contents
1. [Introduction to RAG](#introduction)
2. [Core Components](#core-components)
3. [Document Processing Pipeline](#document-processing)
4. [Embedding Generation](#embedding-generation)
5. [Vector Storage & Retrieval](#vector-storage)
6. [Query Processing](#query-processing)
7. [Response Generation](#response-generation)
8. [Complete RAG Implementation](#complete-implementation)
9. [Evaluation & Metrics](#evaluation)
10. [Real-World Applications](#applications)

---

## Introduction to RAG {#introduction}

Retrieval-Augmented Generation (RAG) is a powerful technique that combines the strengths of information retrieval and text generation. Instead of relying solely on the LLM's training data, RAG allows us to:

- **Retrieve** relevant information from a knowledge base
- **Augment** the LLM's context with this information
- **Generate** responses based on both the query and retrieved context

### Why RAG Matters

**Problem**: LLMs have knowledge cutoff dates and can't access real-time information
**Solution**: RAG provides access to up-to-date, domain-specific knowledge
**Result**: More accurate, factual, and contextually relevant responses

### Business Impact
- **Customer Support**: 60% reduction in ticket volume
- **Legal Analysis**: 20+ hours/week saved in document review
- **Knowledge Management**: 95% accuracy in Q&A systems

## Core Components {#core-components}

A RAG system consists of several key components working together:

```mermaid
graph TD
    A[Documents] --> B[Document Processor]
    B --> C[Text Chunker]
    C --> D[Embedding Model]
    D --> E[Vector Database]
    F[User Query] --> G[Query Processor]
    G --> H[Embedding Model]
    H --> I[Vector Search]
    E --> I
    I --> J[Retrieved Context]
    J --> K[LLM]
    F --> K
    K --> L[Generated Response]
```

### 1. Document Processing
- **Input**: Raw documents (PDFs, text files, web pages)
- **Output**: Cleaned, structured text chunks
- **Key Tasks**: Text extraction, cleaning, chunking

### 2. Embedding Generation
- **Purpose**: Convert text to numerical vectors
- **Models**: Sentence Transformers, OpenAI embeddings
- **Output**: High-dimensional vectors representing semantic meaning

### 3. Vector Storage
- **Purpose**: Store and index embeddings for fast retrieval
- **Options**: Qdrant, Chroma, Pinecone, Weaviate
- **Features**: Similarity search, filtering, metadata

### 4. Query Processing
- **Input**: User question
- **Process**: Convert to embedding, search similar vectors
- **Output**: Relevant document chunks

### 5. Response Generation
- **Input**: Query + Retrieved context
- **Process**: LLM generates response using both
- **Output**: Contextual, accurate answer

In [ ]:
# Install required packages
!pip install -q sentence-transformers qdrant-client redis python-dotenv tiktoken

# Import necessary libraries
import os
import json
import hashlib
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import redis
from dotenv import load_dotenv
import tiktoken

# Import our LLM provider utilities
import sys
sys.path.append('../../utils')
from llm_providers import get_available_providers, LLMProviderFactory, LLMConfig

# Load environment variables
load_dotenv()

# Initialize LLM provider (will use first available provider)
available_providers = get_available_providers()
if not available_providers:
    raise ValueError("No LLM providers available! Please check your .env file and API keys.")

# Use the first available provider
provider_name = list(available_providers.keys())[0]
llm_provider = available_providers[provider_name]

print(f"✅ All packages imported successfully!")
print(f"🔧 Using LLM provider: {provider_name.upper()}")
print(f"🌐 Available providers: {list(available_providers.keys())}")
print("🔧 Environment configured for RAG implementation")

## Document Processing Pipeline {#document-processing}

The first step in any RAG system is processing raw documents into a format suitable for embedding and retrieval.

### Sample Knowledge Base: E-commerce Product Catalog

Let's work with a realistic e-commerce scenario - building a product recommendation and Q&A system.

In [ ]:
# Sample e-commerce product data
sample_products = [
    {
        "id": "laptop-001",
        "name": "MacBook Pro 16-inch M3 Max",
        "category": "Laptops",
        "price": 3999.99,
        "description": "Powerful laptop with M3 Max chip, 32GB RAM, 1TB SSD. Perfect for professional video editing, 3D rendering, and software development. Features Liquid Retina XDR display with ProMotion technology.",
        "specs": {
            "processor": "Apple M3 Max",
            "memory": "32GB unified memory",
            "storage": "1TB SSD",
            "display": "16.2-inch Liquid Retina XDR",
            "graphics": "38-core GPU",
            "battery": "Up to 22 hours",
            "ports": "3x Thunderbolt 4, HDMI, SDXC, MagSafe 3"
        },
        "features": ["Professional performance", "Long battery life", "High-resolution display", "Silent operation"],
        "target_audience": "Professional developers, video editors, graphic designers"
    },
    {
        "id": "laptop-002", 
        "name": "Dell XPS 13 Plus",
        "category": "Laptops",
        "price": 1299.99,
        "description": "Ultra-thin laptop with 13th Gen Intel Core i7, 16GB RAM, 512GB SSD. Premium design with edge-to-edge keyboard and haptic touchpad. Ideal for business professionals and content creators.",
        "specs": {
            "processor": "Intel Core i7-1360P",
            "memory": "16GB LPDDR5",
            "storage": "512GB PCIe SSD",
            "display": "13.4-inch 3.5K OLED",
            "graphics": "Intel Iris Xe",
            "battery": "Up to 12 hours",
            "ports": "2x Thunderbolt 4, 1x USB-C"
        },
        "features": ["Ultra-thin design", "OLED display", "Premium build quality", "Fast performance"],
        "target_audience": "Business professionals, content creators, students"
    },
    {
        "id": "phone-001",
        "name": "iPhone 15 Pro Max",
        "category": "Smartphones", 
        "price": 1199.99,
        "description": "Latest iPhone with A17 Pro chip, 256GB storage, titanium design. Features advanced camera system with 5x optical zoom, Action Button, and USB-C connectivity.",
        "specs": {
            "processor": "A17 Pro chip",
            "memory": "8GB RAM",
            "storage": "256GB",
            "display": "6.7-inch Super Retina XDR",
            "camera": "48MP main, 12MP ultra-wide, 12MP telephoto",
            "battery": "Up to 29 hours video playback",
            "connectivity": "5G, Wi-Fi 6E, Bluetooth 5.3"
        },
        "features": ["Titanium design", "Advanced camera system", "USB-C", "Action Button"],
        "target_audience": "Photography enthusiasts, power users, professionals"
    },
    {
        "id": "headphone-001",
        "name": "Sony WH-1000XM5",
        "category": "Audio",
        "price": 399.99,
        "description": "Industry-leading noise-canceling headphones with 30-hour battery life. Features LDAC codec, speak-to-chat technology, and premium comfort for all-day listening.",
        "specs": {
            "driver": "30mm dynamic",
            "frequency": "4Hz-40kHz",
            "battery": "30 hours (NC on), 40 hours (NC off)",
            "charging": "3 min = 3 hours playback",
            "connectivity": "Bluetooth 5.2, NFC, 3.5mm jack",
            "weight": "250g",
            "features": ["Industry-leading ANC", "LDAC codec", "Speak-to-chat", "Quick attention mode"]
        },
        "features": ["Industry-leading noise cancellation", "30-hour battery", "Premium sound quality", "Comfortable design"],
        "target_audience": "Audiophiles, frequent travelers, remote workers"
    }
]

print(f"📦 Loaded {len(sample_products)} sample products")
print("🎯 Categories:", set(product["category"] for product in sample_products))

In [ ]:
@dataclass
class DocumentChunk:
    """Represents a chunk of text with metadata"""
    id: str
    content: str
    metadata: Dict[str, Any]
    embedding: List[float] = None

class DocumentProcessor:
    """Handles document processing and chunking"""
    
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.encoding = tiktoken.get_encoding("cl100k_base")
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text using tiktoken"""
        return len(self.encoding.encode(text))
    
    def create_product_document(self, product: Dict[str, Any]) -> str:
        """Convert product data to a searchable document"""
        doc_parts = [
            f"Product: {product['name']}",
            f"Category: {product['category']}",
            f"Price: ${product['price']}",
            f"Description: {product['description']}",
            f"Target Audience: {product['target_audience']}",
            f"Key Features: {', '.join(product['features'])}",
            f"Specifications: {', '.join([f'{k}: {v}' for k, v in product['specs'].items()])}"
        ]
        return "\n".join(doc_parts)
    
    def chunk_text(self, text: str, metadata: Dict[str, Any]) -> List[DocumentChunk]:
        """Split text into overlapping chunks"""
        chunks = []
        words = text.split()
        
        start = 0
        chunk_id = 0
        
        while start < len(words):
            # Get chunk of words
            end = min(start + self.chunk_size, len(words))
            chunk_words = words[start:end]
            chunk_text = " ".join(chunk_words)
            
            # Create chunk ID
            chunk_id_str = f"{metadata.get('product_id', 'unknown')}_chunk_{chunk_id}"
            
            # Create chunk
            chunk = DocumentChunk(
                id=chunk_id_str,
                content=chunk_text,
                metadata=metadata.copy()
            )
            chunks.append(chunk)
            
            # Move start position with overlap
            start = end - self.chunk_overlap
            chunk_id += 1
            
            # Prevent infinite loop
            if start >= len(words) - self.chunk_overlap:
                break
        
        return chunks
    
    def process_products(self, products: List[Dict[str, Any]]) -> List[DocumentChunk]:
        """Process all products into chunks"""
        all_chunks = []
        
        for product in products:
            # Create document
            doc = self.create_product_document(product)
            
            # Create metadata
            metadata = {
                "product_id": product["id"],
                "product_name": product["name"],
                "category": product["category"],
                "price": product["price"],
                "source": "ecommerce_catalog"
            }
            
            # Chunk document
            chunks = self.chunk_text(doc, metadata)
            all_chunks.extend(chunks)
        
        return all_chunks

# Initialize processor
processor = DocumentProcessor(chunk_size=300, chunk_overlap=50)

# Process products into chunks
chunks = processor.process_products(sample_products)

print(f"📄 Created {len(chunks)} document chunks")
print(f"📊 Average chunk length: {np.mean([len(chunk.content) for chunk in chunks]):.0f} characters")
print(f"🔍 Sample chunk:")
print(f"ID: {chunks[0].id}")
print(f"Content: {chunks[0].content[:200]}...")
print(f"Metadata: {chunks[0].metadata}")

## Embedding Generation {#embedding-generation}

Embeddings convert text into numerical vectors that capture semantic meaning. Similar texts will have similar embeddings, enabling semantic search.

### Key Concepts:
- **Dimensionality**: Higher dimensions = more detail but more storage/compute
- **Model Choice**: Different models excel at different tasks
- **Normalization**: Ensures consistent vector magnitudes

In [ ]:
class EmbeddingGenerator:
    """Handles text embedding generation"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize embedding model
        
        Popular models:
        - all-MiniLM-L6-v2: Fast, good for general purpose (384 dims)
        - all-mpnet-base-v2: Better quality, slower (768 dims)
        - sentence-transformers/all-MiniLM-L12-v2: Balanced (384 dims)
        """
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
        print(f"✅ Loaded embedding model: {model_name}")
        print(f"📏 Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
    
    def generate_embedding(self, text: str) -> List[float]:
        """Generate embedding for a single text"""
        embedding = self.model.encode(text, convert_to_tensor=False)
        return embedding.tolist()
    
    def generate_embeddings_batch(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for multiple texts efficiently"""
        embeddings = self.model.encode(texts, convert_to_tensor=False)
        return embeddings.tolist()
    
    def normalize_embedding(self, embedding: List[float]) -> List[float]:
        """Normalize embedding to unit vector"""
        embedding_array = np.array(embedding)
        norm = np.linalg.norm(embedding_array)
        if norm == 0:
            return embedding
        return (embedding_array / norm).tolist()

# Initialize embedding generator
embedder = EmbeddingGenerator("all-MiniLM-L6-v2")

# Generate embeddings for all chunks
print("🔄 Generating embeddings for all chunks...")
chunk_texts = [chunk.content for chunk in chunks]
embeddings = embedder.generate_embeddings_batch(chunk_texts)

# Add embeddings to chunks
for chunk, embedding in zip(chunks, embeddings):
    chunk.embedding = embedder.normalize_embedding(embedding)

print(f"✅ Generated {len(embeddings)} embeddings")
print(f"📊 Embedding dimension: {len(embeddings[0])}")
print(f"🔍 Sample embedding (first 5 values): {embeddings[0][:5]}")

# Test semantic similarity
test_queries = [
    "laptop for video editing",
    "wireless headphones with noise cancellation", 
    "expensive smartphone with good camera"
]

print("\n🧪 Testing semantic similarity:")
for query in test_queries:
    query_embedding = embedder.normalize_embedding(embedder.generate_embedding(query))
    
    # Calculate similarities
    similarities = []
    for chunk in chunks:
        similarity = np.dot(query_embedding, chunk.embedding)
        similarities.append((similarity, chunk.metadata['product_name'], chunk.content[:100]))
    
    # Sort by similarity
    similarities.sort(reverse=True)
    
    print(f"\nQuery: '{query}'")
    print("Top matches:")
    for i, (sim, name, content) in enumerate(similarities[:2]):
        print(f"  {i+1}. {name} (similarity: {sim:.3f})")
        print(f"     {content}...")

## Vector Storage & Retrieval {#vector-storage}

Vector databases enable fast similarity search over large collections of embeddings. We'll use Qdrant for this example.

### Vector Database Features:
- **Similarity Search**: Find most similar vectors quickly
- **Filtering**: Combine vector search with metadata filters
- **Scalability**: Handle millions of vectors efficiently
- **Persistence**: Store vectors permanently

In [ ]:
class VectorStore:
    """Handles vector storage and retrieval using Qdrant"""
    
    def __init__(self, collection_name: str = "ecommerce_products"):
        self.collection_name = collection_name
        
        # Get Qdrant configuration from environment
        qdrant_url = os.getenv("QDRANT_URL", "http://localhost:6333")
        qdrant_api_key = os.getenv("QDRANT_API_KEY")
        
        # Initialize Qdrant client
        if qdrant_api_key:
            self.client = QdrantClient(
                url=qdrant_url,
                api_key=qdrant_api_key
            )
        else:
            # For development, use in-memory client
            self.client = QdrantClient(":memory:")
            print("⚠️  Using in-memory Qdrant client. Data will not persist between sessions.")
            print("   To use persistent storage, set QDRANT_URL and QDRANT_API_KEY in your .env file")
        
        self.embedding_dim = 384  # all-MiniLM-L6-v2 dimension
        
    def create_collection(self):
        """Create a new collection in the vector database"""
        try:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.embedding_dim,
                    distance=Distance.COSINE
                )
            )
            print(f"✅ Created collection: {self.collection_name}")
        except Exception as e:
            # Collection might already exist
            print(f"ℹ️  Collection {self.collection_name} already exists or error: {e}")
    
    def add_documents(self, chunks: List[DocumentChunk]):
        """Add document chunks to the vector database"""
        points = []
        
        for chunk in chunks:
            point = PointStruct(
                id=hash(chunk.id) % (2**63),  # Convert to int64
                vector=chunk.embedding,
                payload={
                    "content": chunk.content,
                    "metadata": chunk.metadata,
                    "chunk_id": chunk.id
                }
            )
            points.append(point)
        
        # Batch insert points
        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )
        print(f"✅ Added {len(points)} documents to vector store")
    
    def search(self, query_embedding: List[float], limit: int = 5, 
               filter_conditions: Dict = None) -> List[Dict]:
        """Search for similar documents"""
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=limit,
            query_filter=filter_conditions
        )
        
        results = []
        for hit in search_result:
            results.append({
                "score": hit.score,
                "content": hit.payload["content"],
                "metadata": hit.payload["metadata"],
                "chunk_id": hit.payload["chunk_id"]
            })
        
        return results
    
    def get_collection_info(self):
        """Get information about the collection"""
        try:
            info = self.client.get_collection(self.collection_name)
            return {
                "vectors_count": info.vectors_count,
                "indexed_vectors_count": info.indexed_vectors_count,
                "points_count": info.points_count
            }
        except Exception as e:
            return {"error": f"Could not get collection info: {e}"}

# Initialize vector store
vector_store = VectorStore()

# Create collection
vector_store.create_collection()

# Add all chunks to vector store
vector_store.add_documents(chunks)

# Get collection info
info = vector_store.get_collection_info()
print(f"📊 Collection info: {info}")

# Test search functionality
print("\n🔍 Testing vector search:")
test_query = "laptop for professional video editing"
query_embedding = embedder.generate_embedding(test_query)

results = vector_store.search(query_embedding, limit=3)

print(f"Query: '{test_query}'")
print("Top results:")
for i, result in enumerate(results):
    print(f"\n{i+1}. Score: {result['score']:.3f}")
    print(f"   Product: {result['metadata']['product_name']}")
    print(f"   Category: {result['metadata']['category']}")
    print(f"   Content: {result['content'][:150]}...")

## Query Processing {#query-processing}

The query processing component handles user questions and retrieves relevant context. This is where we implement search strategies and result ranking.

### Query Processing Strategies:
1. **Direct Search**: Use query as-is
2. **Query Expansion**: Add synonyms and related terms
3. **Query Rewriting**: Restructure for better retrieval
4. **Hybrid Search**: Combine vector and keyword search

In [ ]:
class QueryProcessor:
    """Handles query processing and retrieval strategies"""
    
    def __init__(self, vector_store: VectorStore, embedder: EmbeddingGenerator):
        self.vector_store = vector_store
        self.embedder = embedder
        
        # Query expansion patterns for e-commerce
        self.expansion_patterns = {
            "laptop": ["notebook", "computer", "PC", "workstation"],
            "phone": ["smartphone", "mobile", "device"],
            "headphone": ["headset", "earphone", "audio", "headphones"],
            "cheap": ["affordable", "budget", "inexpensive", "low-cost"],
            "expensive": ["premium", "high-end", "luxury", "costly"],
            "good": ["excellent", "great", "quality", "reliable"],
            "fast": ["quick", "rapid", "speedy", "high-performance"]
        }
    
    def expand_query(self, query: str) -> str:
        """Expand query with synonyms and related terms"""
        words = query.lower().split()
        expanded_words = []
        
        for word in words:
            expanded_words.append(word)
            if word in self.expansion_patterns:
                expanded_words.extend(self.expansion_patterns[word][:2])  # Add top 2 synonyms
        
        return " ".join(expanded_words)
    
    def process_query(self, query: str, strategy: str = "semantic", 
                     limit: int = 5, filters: Dict = None) -> List[Dict]:
        """
        Process user query and retrieve relevant documents
        
        Strategies:
        - semantic: Use semantic similarity only
        - expanded: Use query expansion
        - hybrid: Combine semantic + keyword matching
        """
        
        if strategy == "expanded":
            expanded_query = self.expand_query(query)
            print(f"🔍 Expanded query: '{expanded_query}'")
            query_embedding = self.embedder.generate_embedding(expanded_query)
        else:
            query_embedding = self.embedder.generate_embedding(query)
        
        # Retrieve documents
        results = self.vector_store.search(
            query_embedding=query_embedding,
            limit=limit,
            filter_conditions=filters
        )
        
        # Post-process results
        processed_results = []
        for result in results:
            processed_result = {
                "content": result["content"],
                "metadata": result["metadata"],
                "relevance_score": result["score"],
                "chunk_id": result["chunk_id"]
            }
            processed_results.append(processed_result)
        
        return processed_results
    
    def get_context_for_llm(self, query: str, max_chunks: int = 3) -> str:
        """Get formatted context for LLM input"""
        results = self.process_query(query, limit=max_chunks)
        
        context_parts = []
        for i, result in enumerate(results, 1):
            context_parts.append(
                f"Document {i}:\n"
                f"Product: {result['metadata']['product_name']}\n"
                f"Category: {result['metadata']['category']}\n"
                f"Content: {result['content']}\n"
                f"Relevance: {result['relevance_score']:.3f}\n"
            )
        
        return "\n".join(context_parts)

# Initialize query processor
query_processor = QueryProcessor(vector_store, embedder)

# Test different query strategies
test_queries = [
    "What's the best laptop for video editing?",
    "I need cheap wireless headphones",
    "Show me expensive phones with good cameras"
]

print("🧪 Testing query processing strategies:")
for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: '{query}'")
    
    # Test semantic search
    print("\n📊 Semantic Search Results:")
    results = query_processor.process_query(query, strategy="semantic", limit=2)
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result['metadata']['product_name']} (score: {result['relevance_score']:.3f})")
    
    # Test expanded search
    print("\n🔍 Expanded Search Results:")
    results = query_processor.process_query(query, strategy="expanded", limit=2)
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result['metadata']['product_name']} (score: {result['relevance_score']:.3f})")

## Response Generation {#response-generation}

The final step combines the user query with retrieved context to generate a comprehensive, accurate response using an LLM.

### Response Generation Best Practices:
1. **Clear Context**: Provide relevant background information
2. **Structured Output**: Use consistent formatting
3. **Source Attribution**: Reference where information came from
4. **Confidence Indication**: Show certainty levels when appropriate

In [ ]:
class ResponseGenerator:
    """Handles response generation using LLM provider abstraction"""
    
    def __init__(self, llm_provider=None):
        self.llm_provider = llm_provider or llm_provider
        
    def create_system_prompt(self) -> str:
        """Create system prompt for the LLM"""
        return """You are an expert e-commerce assistant helping customers find the right products. 

Your role:
- Analyze product information and customer queries
- Provide accurate, helpful recommendations
- Be specific about product features and benefits
- Always base your answers on the provided context
- If information is not available in the context, say so clearly

Guidelines:
- Be conversational but professional
- Include specific product names and key features
- Mention prices when relevant
- Explain why a product might be suitable
- Suggest alternatives when appropriate"""

    def create_user_prompt(self, query: str, context: str) -> str:
        """Create user prompt with query and context"""
        return f"""Customer Query: {query}

Relevant Product Information:
{context}

Please provide a helpful response based on the product information above. Include specific product recommendations with reasoning."""

    def generate_response(self, query: str, context: str, 
                         temperature: float = 0.7) -> str:
        """Generate response using LLM provider"""
        try:
            # Create the full prompt
            system_prompt = self.create_system_prompt()
            user_prompt = self.create_user_prompt(query, context)
            full_prompt = f"{system_prompt}\n\n{user_prompt}"
            
            # Generate using the provider
            result = self.llm_provider.generate(
                full_prompt,
                temperature=temperature,
                max_tokens=500
            )
            
            # Extract response based on provider type
            if 'error' in result:
                return f"Error generating response: {result['error']}"
            
            # Handle different response formats
            if 'choices' in result and len(result['choices']) > 0:
                return result['choices'][0]['message']['content'].strip()
            elif 'candidates' in result and len(result['candidates']) > 0:
                return result['candidates'][0]['content']['parts'][0]['text'].strip()
            elif 'response' in result:
                return result['response'].strip()
            else:
                return str(result)
                
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_structured_response(self, query: str, context: str) -> Dict[str, Any]:
        """Generate structured response with metadata"""
        response_text = self.generate_response(query, context)
        
        # Extract product mentions (simple regex-based approach)
        import re
        product_pattern = r'([A-Z][a-zA-Z\s]+(?:Pro|Max|Plus|XPS|MacBook|iPhone|Sony))'
        mentioned_products = re.findall(product_pattern, response_text)
        
        return {
            "response": response_text,
            "mentioned_products": list(set(mentioned_products)),
            "query": query,
            "context_length": len(context),
            "provider": self.llm_provider.config.provider,
            "model": self.llm_provider.config.model
        }

# Initialize response generator
response_generator = ResponseGenerator(llm_provider)

# Test response generation
test_queries = [
    "What's the best laptop for video editing under $2000?",
    "I need wireless headphones with good noise cancellation",
    "Show me the most expensive phone with the best camera"
]

print("🤖 Testing response generation:")
for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    
    # Get context
    context = query_processor.get_context_for_llm(query, max_chunks=3)
    
    # Generate response
    structured_response = response_generator.generate_structured_response(query, context)
    
    print(f"\nResponse:")
    print(structured_response["response"])
    print(f"\nMentioned Products: {structured_response['mentioned_products']}")
    print(f"Context Length: {structured_response['context_length']} characters")
    print(f"Provider: {structured_response['provider']} ({structured_response['model']})")

## Complete RAG Implementation {#complete-implementation}

Now let's put it all together into a complete RAG system that can be used in production.

In [ ]:
class RAGSystem:
    """Complete RAG system integrating all components with caching"""
    
    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 llm_provider=None,
                 chunk_size: int = 300,
                 chunk_overlap: int = 50):
        
        # Initialize components
        self.embedder = EmbeddingGenerator(embedding_model)
        self.processor = DocumentProcessor(chunk_size, chunk_overlap)
        self.vector_store = VectorStore()
        self.query_processor = QueryProcessor(self.vector_store, self.embedder)
        self.response_generator = ResponseGenerator(llm_provider)
        
        # Initialize Redis cache
        self.redis_client = self._init_redis()
        
        # Create collection
        self.vector_store.create_collection()
        
        print("🚀 RAG System initialized successfully!")
    
    def _init_redis(self):
        """Initialize Redis client with fallback"""
        try:
            redis_url = os.getenv("REDIS_URL", "redis://localhost:6379")
            client = redis.from_url(redis_url, decode_responses=True)
            # Test connection
            client.ping()
            print("✅ Redis cache connected")
            return client
        except Exception as e:
            print(f"⚠️  Redis not available: {e}")
            print("   Caching disabled - using in-memory fallback")
            return None
    
    def _get_cache_key(self, query: str, strategy: str, max_chunks: int) -> str:
        """Generate cache key for query"""
        import hashlib
        key_data = f"{query}_{strategy}_{max_chunks}"
        return f"rag:{hashlib.md5(key_data.encode()).hexdigest()}"
    
    def _get_from_cache(self, cache_key: str) -> Dict[str, Any]:
        """Get response from cache"""
        if not self.redis_client:
            return None
        
        try:
            cached = self.redis_client.get(cache_key)
            if cached:
                return json.loads(cached)
        except Exception as e:
            print(f"Cache read error: {e}")
        return None
    
    def _set_cache(self, cache_key: str, response: Dict[str, Any], ttl: int = 3600):
        """Store response in cache"""
        if not self.redis_client:
            return
        
        try:
            self.redis_client.setex(cache_key, ttl, json.dumps(response))
        except Exception as e:
            print(f"Cache write error: {e}")
    
    def add_documents(self, documents: List[Dict[str, Any]]):
        """Add documents to the knowledge base"""
        print(f"📄 Processing {len(documents)} documents...")
        
        # Process documents into chunks
        chunks = self.processor.process_products(documents)
        
        # Generate embeddings
        print("🔄 Generating embeddings...")
        chunk_texts = [chunk.content for chunk in chunks]
        embeddings = self.embedder.generate_embeddings_batch(chunk_texts)
        
        # Add embeddings to chunks
        for chunk, embedding in zip(chunks, embeddings):
            chunk.embedding = self.embedder.normalize_embedding(embedding)
        
        # Store in vector database
        self.vector_store.add_documents(chunks)
        
        print(f"✅ Added {len(chunks)} chunks to knowledge base")
        return len(chunks)
    
    def query(self, question: str, 
              strategy: str = "semantic",
              max_chunks: int = 3,
              temperature: float = 0.7,
              use_cache: bool = True) -> Dict[str, Any]:
        """Query the RAG system with caching"""
        
        # Check cache first
        if use_cache:
            cache_key = self._get_cache_key(question, strategy, max_chunks)
            cached_response = self._get_from_cache(cache_key)
            if cached_response:
                print(f"🎯 Cache hit for query: '{question}'")
                cached_response["cached"] = True
                return cached_response
        
        print(f"🔍 Processing query: '{question}'")
        
        # Get relevant context
        context = self.query_processor.get_context_for_llm(
            question, max_chunks=max_chunks
        )
        
        # Generate response
        structured_response = self.response_generator.generate_structured_response(
            question, context
        )
        
        # Add metadata
        structured_response.update({
            "strategy": strategy,
            "max_chunks": max_chunks,
            "temperature": temperature,
            "timestamp": str(pd.Timestamp.now()),
            "cached": False
        })
        
        # Cache the response
        if use_cache:
            self._set_cache(cache_key, structured_response)
        
        return structured_response
    
    def get_system_stats(self) -> Dict[str, Any]:
        """Get system statistics"""
        collection_info = self.vector_store.get_collection_info()
        cache_stats = {}
        
        if self.redis_client:
            try:
                cache_info = self.redis_client.info('memory')
                cache_stats = {
                    "redis_connected": True,
                    "redis_memory_used": cache_info.get('used_memory_human', 'Unknown'),
                    "redis_keys": self.redis_client.dbsize()
                }
            except:
                cache_stats = {"redis_connected": False}
        else:
            cache_stats = {"redis_connected": False}
        
        return {
            "total_documents": collection_info["points_count"],
            "embedding_model": self.embedder.model_name,
            "llm_provider": self.response_generator.llm_provider.config.provider,
            "llm_model": self.response_generator.llm_provider.config.model,
            "collection_name": self.vector_store.collection_name,
            **cache_stats
        }

# Initialize complete RAG system
rag_system = RAGSystem(llm_provider=llm_provider)

# Add our sample products
rag_system.add_documents(sample_products)

# Get system stats
stats = rag_system.get_system_stats()
print(f"\n📊 System Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

# Test the complete system
print(f"\n{'='*80}")
print("🧪 Testing Complete RAG System")
print(f"{'='*80}")

test_questions = [
    "What's the best laptop for professional video editing?",
    "I need wireless headphones under $500 with excellent noise cancellation",
    "Show me the most expensive smartphone with the best camera system",
    "What laptop would be good for a student on a budget?",
    "I want a phone with long battery life and good performance"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {question}")
    print(f"{'='*60}")
    
    response = rag_system.query(question, strategy="semantic", max_chunks=2)
    
    print(f"\nAnswer:")
    print(response["response"])
    print(f"\nMentioned Products: {response['mentioned_products']}")
    print(f"Context Length: {response['context_length']} characters")
    print(f"Cached: {response.get('cached', False)}")
    print(f"Provider: {response.get('llm_provider', 'Unknown')} ({response.get('llm_model', 'Unknown')})")

## Evaluation & Metrics {#evaluation}

Evaluating RAG systems is crucial for ensuring quality and performance. Let's implement some basic evaluation metrics.

In [ ]:
import time
import pandas as pd
from typing import List, Dict, Any

class RAGEvaluator:
    """Evaluates RAG system performance"""
    
    def __init__(self, rag_system: RAGSystem):
        self.rag_system = rag_system
        
    def evaluate_retrieval_quality(self, queries: List[str], 
                                 expected_products: List[List[str]]) -> Dict[str, float]:
        """Evaluate retrieval quality using precision and recall"""
        
        precisions = []
        recalls = []
        
        for query, expected in zip(queries, expected_products):
            # Get retrieval results
            results = self.rag_system.query_processor.process_query(query, limit=3)
            retrieved_products = [r['metadata']['product_name'] for r in results]
            
            # Calculate precision and recall
            if retrieved_products:
                precision = len(set(retrieved_products) & set(expected)) / len(retrieved_products)
                precisions.append(precision)
            else:
                precisions.append(0.0)
            
            if expected:
                recall = len(set(retrieved_products) & set(expected)) / len(expected)
                recalls.append(recall)
            else:
                recalls.append(0.0)
        
        return {
            "precision": np.mean(precisions),
            "recall": np.mean(recalls),
            "f1_score": 2 * np.mean(precisions) * np.mean(recalls) / (np.mean(precisions) + np.mean(recalls)) if (np.mean(precisions) + np.mean(recalls)) > 0 else 0
        }
    
    def evaluate_response_time(self, queries: List[str]) -> Dict[str, float]:
        """Evaluate response time for queries"""
        
        response_times = []
        
        for query in queries:
            start_time = time.time()
            self.rag_system.query(query)
            end_time = time.time()
            
            response_times.append(end_time - start_time)
        
        return {
            "avg_response_time": np.mean(response_times),
            "min_response_time": np.min(response_times),
            "max_response_time": np.max(response_times),
            "std_response_time": np.std(response_times)
        }
    
    def evaluate_context_relevance(self, queries: List[str]) -> Dict[str, float]:
        """Evaluate how relevant the retrieved context is"""
        
        relevance_scores = []
        
        for query in queries:
            # Get context
            context = self.rag_system.query_processor.get_context_for_llm(query, max_chunks=3)
            
            # Simple relevance check: count query terms in context
            query_terms = set(query.lower().split())
            context_terms = set(context.lower().split())
            
            if query_terms:
                relevance = len(query_terms & context_terms) / len(query_terms)
                relevance_scores.append(relevance)
        
        return {
            "avg_relevance": np.mean(relevance_scores),
            "min_relevance": np.min(relevance_scores),
            "max_relevance": np.max(relevance_scores)
        }
    
    def comprehensive_evaluation(self, test_queries: List[str], 
                               expected_products: List[List[str]]) -> Dict[str, Any]:
        """Run comprehensive evaluation"""
        
        print("🔍 Running comprehensive RAG evaluation...")
        
        # Retrieval quality
        retrieval_metrics = self.evaluate_retrieval_quality(test_queries, expected_products)
        
        # Response time
        response_time_metrics = self.evaluate_response_time(test_queries)
        
        # Context relevance
        relevance_metrics = self.evaluate_context_relevance(test_queries)
        
        # Combine all metrics
        evaluation_results = {
            "retrieval_quality": retrieval_metrics,
            "response_time": response_time_metrics,
            "context_relevance": relevance_metrics,
            "overall_score": (
                retrieval_metrics["f1_score"] * 0.4 +
                (1 - min(response_time_metrics["avg_response_time"] / 5, 1)) * 0.3 +
                relevance_metrics["avg_relevance"] * 0.3
            )
        }
        
        return evaluation_results

# Test evaluation
evaluator = RAGEvaluator(rag_system)

# Test queries with expected products
test_queries = [
    "What's the best laptop for video editing?",
    "I need wireless headphones with noise cancellation",
    "Show me expensive smartphones with good cameras"
]

expected_products = [
    ["MacBook Pro 16-inch M3 Max"],  # Expected for video editing
    ["Sony WH-1000XM5"],  # Expected for noise cancellation
    ["iPhone 15 Pro Max"]  # Expected for expensive phone with good camera
]

# Run evaluation
evaluation_results = evaluator.comprehensive_evaluation(test_queries, expected_products)

print("\n📊 Evaluation Results:")
print("="*50)

print("\n🎯 Retrieval Quality:")
for metric, value in evaluation_results["retrieval_quality"].items():
    print(f"  {metric}: {value:.3f}")

print("\n⏱️ Response Time:")
for metric, value in evaluation_results["response_time"].items():
    print(f"  {metric}: {value:.3f}s")

print("\n🎯 Context Relevance:")
for metric, value in evaluation_results["context_relevance"].items():
    print(f"  {metric}: {value:.3f}")

print(f"\n🏆 Overall Score: {evaluation_results['overall_score']:.3f}")

# Performance interpretation
overall_score = evaluation_results['overall_score']
if overall_score >= 0.8:
    performance_level = "Excellent"
elif overall_score >= 0.6:
    performance_level = "Good"
elif overall_score >= 0.4:
    performance_level = "Fair"
else:
    performance_level = "Needs Improvement"

print(f"\n📈 Performance Level: {performance_level}")

## Real-World Applications {#applications}

RAG systems are being used across various industries to solve real business problems. Here are some examples:

### 1. Customer Support (60% ticket reduction)
- **Problem**: High volume of repetitive customer questions
- **Solution**: RAG-powered Q&A system with knowledge base
- **Result**: Faster response times, reduced agent workload

### 2. Legal Document Analysis (20+ hours/week saved)
- **Problem**: Manual review of contracts and legal documents
- **Solution**: RAG system trained on legal corpus
- **Result**: Automated clause extraction and risk assessment

### 3. E-commerce Product Recommendations
- **Problem**: Customers can't find relevant products
- **Solution**: RAG system with product catalog and customer queries
- **Result**: Improved product discovery and conversion rates

### 4. Technical Documentation Q&A
- **Problem**: Developers struggle to find relevant documentation
- **Solution**: RAG system with code examples and documentation
- **Result**: Faster onboarding and problem resolution

### 5. Healthcare Knowledge Base
- **Problem**: Medical professionals need quick access to latest research
- **Solution**: RAG system with medical literature and guidelines
- **Result**: Better patient care and treatment decisions

## Key Takeaways & Next Steps

### What We've Built
✅ **Complete RAG System** with document processing, embedding generation, vector storage, and response generation
✅ **Evaluation Framework** for measuring system performance
✅ **Real-world Example** using e-commerce product recommendations

### Key Concepts Learned
1. **Document Processing**: Chunking strategies and metadata management
2. **Embeddings**: Converting text to numerical vectors for semantic search
3. **Vector Storage**: Efficient storage and retrieval of embeddings
4. **Query Processing**: Different strategies for handling user queries
5. **Response Generation**: Combining context with LLM for accurate answers

### Next Steps
- **Advanced RAG**: Hybrid search, reranking, and query expansion
- **Production Deployment**: Scaling, monitoring, and optimization
- **Evaluation**: More sophisticated metrics and testing frameworks
- **Specialized Applications**: Domain-specific RAG implementations

### Performance Optimization Tips
1. **Chunk Size**: Experiment with different sizes (200-500 tokens)
2. **Embedding Models**: Choose based on your domain and language
3. **Vector Database**: Consider scalability and feature requirements
4. **Query Processing**: Implement query expansion and rewriting
5. **Caching**: Cache frequent queries and embeddings

---

**Ready to build your own RAG system?** Start with a small dataset, implement the basic components, and gradually add advanced features based on your specific use case!